In [72]:
import numpy as np
from dotenv import load_dotenv
from dwave.system import DWaveSampler, EmbeddingComposite
from numpy import hamming
from openjij import SQASampler
from pyqubo import Array, Constraint, LogEncInteger, solve_qubo
import math
import os
import dimod

In [73]:
load_dotenv()

TOKEN = os.getenv("TOKEN")
endpoint = "https://cloud.dwavesys.com/sapi/"

In [74]:
W = 750
cost = [135, 139, 149, 150, 156, 163, 173,
        184, 192, 201, 210, 214, 221, 229, 240]
weight = [70, 73, 77, 80, 82, 87, 90, 94, 98, 106, 110, 113, 115, 118, 120]
opt = [1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1]
c = {}
w = {}
N = len(cost)
for i in range(N):
    c[i] = cost[i]
    w[i] = weight[i]

In [75]:
x = Array.create('x', shape=(N), vartype='BINARY')
y = LogEncInteger("y", (0, W))

In [76]:
key1 = max(c, key=lambda k: c[k])
B = 1
A = B * c[key1] * 5

In [77]:
HA = Constraint(
    A * (W - sum(w[a] * x[a] for a in range(N)) - y)**2, label='HA'
)

HB = - B * sum(c[a] * x[a] for a in range(N))

In [78]:
H = HA + HB
model = H.compile()
Q, offset = model.to_qubo()

In [79]:
dw_sampler = DWaveSampler(solver='DW_2000Q_6',
                          token=TOKEN, endpoint=endpoint)
sampler = EmbeddingComposite(dw_sampler)
sampleset = sampler.sample_qubo(Q, num_reads=30)

In [80]:
sampleset.record

rec.array([([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1], -5.96970697e+08, 1, 0.16),
           ([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1], -5.95123897e+08, 1, 0.16),
           ([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1], -5.94503497e+08, 1, 0.12),
           ([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1], -5.95741897e+08, 1, 0.12),
           ([0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1], -5.44320846e+08, 1, 0.12),
           ([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1], -5.89453897e+08, 1, 0.16),
           ([0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1], -5.55173646e+08, 1, 0.12),
           ([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1], -5.42732047e+08, 1, 0.16),
           ([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,

In [81]:
decoded_sample = model.decode_sample(sampleset.first.sample, vartype="BINARY")

In [82]:
cost = 0
weight = 0
for k in range(N):
    if decoded_sample.array('x', k) != 0:
        cost += c[k]
        weight += w[k]
        

In [83]:
sol_y = sum(2**k * v for k, v in [(elem, decoded_sample.array('y', elem))
            for elem in range(math.ceil(math.log2(W)))])

In [84]:
print("スラック変数Y = {}".format(sol_y))
print()
print("broken")
print(decoded_sample.constraints(only_broken=True))
print("cost : "+str(cost))
print("weight : "+str(weight))

スラック変数Y = 32

broken
{'HA': (False, 7680000.0)}
cost : 1539
weight : 798


In [85]:
init = sampleset.samples()[-1]

In [86]:
t = (0, 5, 15, 20)
s = (1.0, 0.4, 0.4, 1.0)

schedule = list(zip(t, s))

In [87]:
dw_sampler = DWaveSampler(solver='DW_2000Q_6',
                          token=TOKEN, endpoint=endpoint)
sampler = EmbeddingComposite(dw_sampler)
sampleset = sampler.sample_qubo(
    Q, num_reads=30, anneal_schedule=schedule, initial_state=init)

In [88]:
sampleset.record

rec.array([([0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1], -5.31341562e+08, 1, 0.08),
           ([0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1], -5.19480762e+08, 1, 0.08),
           ([0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1], -5.19480762e+08, 1, 0.08),
           ([0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1], -5.26315962e+08, 1, 0.08),
           ([0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1], -4.40564118e+08, 1, 0.16),
           ([0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], -4.90603962e+08, 1, 0.08),
           ([0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1], -4.61300118e+08, 1, 0.08),
           ([1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0], -6.74798263e+08, 1, 0.08),
           ([1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,

In [89]:
decoded_sample = model.decode_sample(sampleset.first.sample, vartype="BINARY")

In [90]:
cost = 0
weight = 0
for k in range(N):
    if decoded_sample.array('x', k) != 0:
        cost += c[k]
        weight += w[k]

In [91]:
sol_y = sum(2**k * v for k, v in [(elem, decoded_sample.array('y', elem))
            for elem in range(math.ceil(math.log2(W)))])

In [92]:
print("スラック変数Y = {}".format(sol_y))
print()
print("broken")
print(decoded_sample.constraints(only_broken=True))
print("cost : "+str(cost))
print("weight : "+str(weight))

スラック変数Y = 212

broken
{'HA': (False, 202800.0)}
cost : 1063
weight : 551
